<style>
.jp-RenderedHTMLCommon pre {
        background-color: #FFFFF0FF;
    }
</style>

# Distributed Systems
## 2021/22

Lab 5

Nuno Preguiça, Sérgio Duarte, Dina Borrego, João Vilalonga

# Goals

In the end of this lab you should be able to:

+ Understand what a Web Service SOAP is
+ Know how to develop a WS SOAP and Server in Java
+ Know how to develop a SOAP Client in Java
+ Understand xxx

# SOAP : Simple Object Access Protocol

A Messaging protocol specification for exchanging structured
data between web services.

+ Specially tailored for supporting machine-to-machine inter-operation and interaction.
+ Data is represented in XML (which is more verbose than Json for instance).
+ Operates on top of application-level protocols, usually HTTP.

# SOAP WSDL - Interface Description Language for Web Services

Every Web Service SOAP exposes their interface through a machine processable format named WSDL.

You can check the WSDL specification in your browser using a link with this format: 

`http:/machine:port/ServicePath/?wsdl`



# SOAP WSDL - Interface Description Language for Web Services

The WSDL document specifies:

+ The operations exposed by the web service.
+ The message (and their format) used to execute each operation (including the format of arguments and return values)
+ Specifies how and where the service can be reached.

A detailed explanation of the structure of WSDL files can be found [here](https://www.w3schools.com/xml/xml_wsdl.asp).

```xml
<definitions xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" xmlns:wsp="http://www.w3.org/ns/ws-policy" xmlns:wsp1_2="http://schemas.xmlsoap.org/ws/2004/09/policy" xmlns:wsam="http://www.w3.org/2007/05/addressing/metadata" xmlns:soap="http://schemas.xmlsoap.org/wsdl/soap/" xmlns:tns="http://sd2122" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns="http://schemas.xmlsoap.org/wsdl/" targetNamespace="http://sd2122" name="users">
<types>
<xsd:schema>
<xsd:import namespace="http://sd2122" schemaLocation="http://192.168.100.2:13456/soap?xsd=1"/>
</xsd:schema>
</types>
<message name="createUser">
<part name="parameters" element="tns:createUser"/>
</message>
<message name="UsersException">
<part name="fault" element="tns:UsersException"/>
</message>
<portType name="SoapUsers">
<operation name="createUser">
<input wsam:Action="http://sd2122/SoapUsers/createUserRequest" message="tns:createUser"/>
<output wsam:Action="http://sd2122/SoapUsers/createUserResponse" message="tns:createUserResponse"/>
<fault message="tns:UsersException" name="UsersException" wsam:Action="http://sd2122/SoapUsers/createUser/Fault/UsersException"/>
</operation>
</portType>
<binding name="SoapUsersWebServicePortBinding" type="tns:SoapUsers">
<soap:binding transport="http://schemas.xmlsoap.org/soap/http" style="document"/>
<operation name="createUser">
<soap:operation soapAction=""/>
<input>
<soap:body use="literal"/>
</input>
<output>
<soap:body use="literal"/>
</output>
<fault name="UsersException">
<soap:fault name="UsersException" use="literal"/>
</fault>
</operation>
</binding>
<service name="users">
<port name="SoapUsersWebServicePort" binding="tns:SoapUsersWebServicePortBinding">
<soap:address location="http://192.168.100.2:13456/soap"/>
</port>
</service>
</definitions>
```

# SOAP Architecture

<img src="https://preguica.github.io/sd2122/praticas2122/aula5/soap-architecture.png" width="75%" ><img>

The WSDL document is fetched by the client prior to the first request to the server.

The client performs service requests by exchanging messages using SOAP XML protocol, usually over
HTTP, but SMTP can also be used.

# SOAP Interaction

SOAP Web Services are not tied to a particular programming language...

WDSL XML document only describes the service, in terms of available operations, parameters, results and faults. 

It is designed to be parsed by machines to generate client-side stubs automatically.

In Java, the JAX-WS framework can be used to develop SOAP clients
and servers with little effort.
 + Client-side, a server proxy is generated automatically from the WSDL document of a running service instance;
 + Server-side, the stub is generated automatically from Java annonations.
 

# SOAP – Additional Aspects

Parameters and return values in SOAP are passed by ***value***.

Types of parameters and return values can be of different on the client side,

Example: `String[] -> List<String>`

Common, when different programming languages are involved, or depending
on the tools used to generate the stubs automatically.


Note:

Passing references between clients and servers and vice-versa, is **not possible**...

Clients and servers do not have to be implemented in the same programming language;

Client and server architectures can be totally different and represent data differently.

# SOAP Web Services in Java

## Server-side

Service API, modelled as a **Java interface**;

Service logic, implemented in a **Java class**.

Java annotations to expose:
+ The class that implements the service;
+ Which methods are made available to clients;
+ Exceptions that might be generated by methods exposed by the service;
+ Namespace information (to avoid name clashes with other Web Services)

## Server-side - Users WebService Example


```java
@WebService(serviceName=SoapUsers.NAME, 
            targetNamespace=SoapUsers.NAMESPACE, 
            endpointInterface=SoapUsers.INTERFACE)
public interface SoapUsers {

	static final String NAME = "users";
	static final String NAMESPACE = "http://sd2122";
	static final String INTERFACE = "sd2122.aula5.api.service.soap.SoapUsers";

	@WebMethod
	String createUser(User user) throws UsersException;
    ...
```

Standard Java Interface enriched with annotations and identifying the methods supported by your service.


### @WebService

Used to expose an interface and its implementation class as a remotely
accessible Soap WebService.

`serviceName` - the name of the webservice;

`targetNamespace` - used to fully qualify webservice names to avoid clashes with other webservices;

`endpointInterface` - name of the Java interface to instanciate, when the client is Java-based.

### @WebMethod

Exposes which methods of the service interface are acessible remotely.

### @WebFault

In Soap, faults and errors are modelled as exceptions in the service API.

Exceptions thrown in methods that are part of the service API need to be annotated with `@WebFault`

```java
@WebFault
public class UsersException extends Exception {

	public UsersException() {
		super("");
	}

	public UsersException(String errorMessage ) {
		super(errorMessage);
	}
```

## Service Implementation

The implementation class needs to **repeat** the (same) `@WebService`annotation.

`@WebMethod` annotations can be omitted.

```java
@WebService(serviceName=SoapUsers.NAME, targetNamespace=SoapUsers.NAMESPACE, endpointInterface=SoapUsers.INTERFACE)
public class SoapUsersWebService implements SoapUsers {

	@Override
	public String createUser(User user) throws UsersException {
        ...
    }
...
```

Unlike JAX-RS, the implementation class ***does not require*** a public no-args constructor.

Methods are normal Java methods...

## Service Instantiation - embedded server

The simplest way to instantiate a SOAP webservice is to use the [Endpoint](https://jakarta.ee/specifications/platform/9/apidocs/jakarta/xml/ws/endpoint) class to publish the service
to a dedicated HTTP server.

```java
var serverBaseURI = "http://host:port/service";
Endpoint.publish(serverBaseURI, new SoapUsersWebService());
```


## Service Instantiation - external server

It is also possible to deploy SOAP web services in an existing HTTP server.

This method will be used and explained later, to allow TLS
secure HTTPS communications to be used in the second lab assignment.

### Server Debugging

Set the following properties before the service is published to log incoming requests.

```java
System.setProperty("com.sun.xml.ws.transport.http.client.HttpTransportPipe.dump", "true");
System.setProperty("com.sun.xml.internal.ws.transport.http.client.HttpTransportPipe.dump", "true");
System.setProperty("com.sun.xml.ws.transport.http.HttpAdapter.dump", "true");
System.setProperty("com.sun.xml.internal.ws.transport.http.HttpAdapter.dump", "true");
```

## Client-side - Stub generation

Given a URL to an instance of the service, the following code automatically generates
a *stub* object that implements the SoapUsers interface.

```java
QName qname = new QName(SoapUsers.NAMESPACE, SoapUsers.NAME);		
Service service = Service.create( URI.create(serverUrl + "?wsdl").toURL(), qname);		
SoapUsers users = service.getPort(sd2122.aula5.api.service.soap.SoapUsers.class);
```

## Client-side - Remote method invocation

Invoking methods on the `users` *stub*, produces remote invocations of those methods on the service pointed
by the URL used to generate the *stub*.

```java
...
SoapUsers users = service.getPort(sd2122.aula5.api.service.soap.SoapUsers.class);

User u = new User( userId, fullName, email, password);

try {
    var result = users.createUser( u );
    System.out.println("Result: " + result);
} catch( UsersException x ) {
    // handle error
}
```

## Client-side - Invocation failures

Client-side invocation failures are dealt by catching `WebServiceException`

```java 
...
SoapUsers users = service.getPort(sd2122.aula5.api.service.soap.SoapUsers.class);

User u = new User( userId, fullName, email, password);

try {
    var result = users.createUser( u );
    System.out.println("Result: " + result);
} catch( UsersException x ) {
    // handle service error
} catch( WebServiceException we) {
    // handle invocation error, maybe retry?
}

```

## Client-side - Connection timeouts

To set the communication timeout values used by the client...

```java
static final int READ_TIMEOUT = 5000;
static final int CONNECT_TIMEOUT = 5000;
...
SoapUsers users = service.getPort(sd2122.aula5.api.service.soap.SoapUsers.class);

((BindingProvider) users).getRequestContext().put(BindingProviderProperties.CONNECT_TIMEOUT, CONNECT_TIMEOUT);
((BindingProvider) users).getRequestContext().put(BindingProviderProperties.REQUEST_TIMEOUT, READ_TIMEOUT);
```



# Programming Considerations

The first project ideally will ***mix*** REST and SOAP services...

## Goals

1. Avoid repeating the same service logic code across servers of different tecnologies;
2. Allow clients to interact with a web service implemented in different technologies.

## Sharing code between a REST and a SOAP web service

Service logic is the **same**, the way results and errors are 
reported back to the caller is **different**.

---

**Solution**: 

1. Model and implement the service logic in a way that abstracts how results and errors are returned to the caller.

2. REST and SOAP versions of the service will share the more abstract implementation.

### Result&lt;T&gt;

Will be used to represent either a (success) value of type T or an error.

```java
public interface Result<T> {

	enum ErrorCode{ OK, CONFLICT, NOT_FOUND, BAD_REQUEST, FORBIDDEN, INTERNAL_ERROR, NOT_IMPLEMENTED};
	
	boolean isOK();
	
	T value();

	ErrorCode error();
	
	static <T> Result<T> ok() {...}

	static <T> Result<T> ok( T result ){...}

	static <T> Result<T> error(ErrorCode error) {...}
}
```

### Abstract Users Service API 

An abstract version of the service API captures the shared service logic...

```java
public interface Users {

	Result<String> createUser(User user);
	
	Result<User> getUser(String userId, String password);
	
	Result<User> updateUser(String userId, String password, User user);
	
	Result<User> deleteUser(String userId, String password);
	
	Result<List<User>> searchUsers(String pattern);	
}
```

### Shared Users Service Logic Implementation 

A regular Java class implements the service logic, producing
the expected behavior in terms of `Result<T>`

```java
public class JavaUsers {
    
    Result<String> createUser(User user) {
        ...
         if(...)
                return Result.error(ErrorCode.CONFLICT);

        return Result.ok( user.userId );
    }
    
}
```

## REST Service implementation 

```java
public class UsersResources implements RestUsers {
    
    final Users impl = new JavaUsers();
    
    public String createUser(User user) {
        var result = impl.createUser( user );
        if( result.isOK() )
            return return result.value();
        else
            throw new WebApplicationException( ... ) ;
    }
    ...
}
```


To complete this idea, we need some additional method to convert the `enum ErrorCode` into an HTTP status code needed
to instantiate the `WebApplicationException`.

Since all methods will just call the implementation and process the result; this can (and should) be handled in a generic super class generic method (like we did for reRry, in a previous class).

## SOAP Service implementation 

```java
public class UsersWebService implements SoapUsers {
    
    final Users impl = new JavaUsers();
    
    public String createUser(User user) throws UsersException {
        var result = impl.createUser( user );
        if( result.isOK() )
            return return result.value();
        else
            throw new UsersException( ... ) ;
    }
    ...
}
```


In this case, we have a way of converting the `enum ErrorCode` into a message to use as the argument of the `UsersException`.

The same observation made above for REST applies, regarding how to process and return the result.

# Interoperable Services

In the project, services will interact with each other. For instance, `Directory` will have
to interact with both `Users` and `Files`.

In a **mixed** REST and SOAP deployment, services will need to be REST clients and SOAP clients
of a different service, in any combination.

### Goal
Avoid code like this all over the methods that need to call other services:
```java
        if( serverURI.endsWith("rest") )
            // REST client code goes here
        else
            // SOAP client code goes here
```

## Client Factories

The [factory method pattern](https://en.wikipedia.org/wiki/Factory_method_pattern) allows to retrieve a REST or SOAP client implementation given the URI of the server.

The factory returns an object that hides the implementation details, in this case the REST or SOAP nature of server.

### REST Client Sketch

```java
public class RestUsersClient implements Users {

    public Result<T> createUser( user ) {
        Response r = target.path(...)
        ...
    }
}
```

Note that we are implementing the abstract version of service `Users`, using Jersey client code.

### SOAP Client Sketch 

```java
public class SoapUsersClient implements Users {

    public Result<T> createUser( user ) {
        SoapUsers users = ...
        ...
    }
}
```

Note that we are implementing the abstract version of service `Users`, using the SOAP client stub.

### ClientFactory Sketch

The factory decides which client implementation to return based on the server URI.

```java
public class UsersClientFactory {
    
    public static Users getClient() {
       var serverURI = ... // use discovery to find a uri of the Users service;
       if( serverURI.endsWith("rest")
          return new RestUsersClient( serverURI );
       else
          return new SoapUsersClient( serverURI );
    }
}
```

Note that the factory returns objects that implement the `Users` interface, either
implementing the REST client, or the SOAP client.

The factory can include a *cache* to avoid discovering and instance the client all the time.

Check [Guava](https://github.com/google/guava) for a library that provides configurable caches.

# Exercises

1. Download Lab 5 project [code]()<br>

2. Create the docker image:
    
    `mvn clean compile assembly:single docker:build`<br>
    
3. Execute the server:
    
    `docker run --rm -t -h users-1 --name users-1 -p 8080:8080 --network sdnet sd2122-aula5-xxxxx-yyyyy`<br>
    
4. Examine the WSDL service specification in the browser:

     [http://localhost:8080/soap/users?wsdl](http://localhost:8080/soap/users?wsdl)<br>
     
5. Execute the SOAP CreateUsersClient

    `docker run -ti --network=sdnet sd2122-aula5-xxxxx-yyyyy /bin/bash`

    `java --add-opens java.base/java.lang=ALL-UNNAMED -cp /home/sd/sd2122.jar sd2122.aula5.clients.soap.CreateUsersClient http://users-1:8080/soap nmp "Nuno Preguiça" nmp@nova.pt 12345`<br>
     
6. Execute again the SOAP CreateUsersClient and a conflict error should be reported.